<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Only do this once per VM, otherwise you'll get multiple clones and nested directories
# !git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
# %cd data-science-from-scratch-Python/
# !pip install import-ipynb

In [0]:
# Chapter 5 - Statistics

# Cheating and copying this list
num_friends = [100.0,49,41,40,25,21,21,19,19,18,18,16,15,15,15,15,14,14,13,13,13,13,12,12,11,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,8,8,8,8,8,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

In [0]:
from collections import Counter
import matplotlib.pyplot as plt

friend_counts = Counter(num_friends)
xs = range(101)
ys = [friend_counts[x] for x in xs]
plt.bar(xs, ys)
plt.axis([0,101,0,25])
plt.title("Histogram of Friend Counts")
plt.xlabel("# of friends")
plt.ylabel("# of people")
plt.show()

In [0]:
num_points = len(num_friends)
largest_value = max(num_friends)
smallest_value = min(num_friends)

sorted_values = sorted(num_friends)
smallest_value = sorted_values[0]
second_smallest_value = sorted_values[1]
second_largest_value = sorted_values[-2]
print(num_points, smallest_value, largest_value, second_smallest_value, second_largest_value)

In [0]:
from typing import List

# Mean
def mean(xs: List[float]) -> float:
  return sum(xs) / len(xs)

In [0]:
mean(num_friends)

In [0]:
# Let's write some median functions. If an odd number in the list, then just the median
# If an even number of things, then the average of the two closest to the median

# The underscores indicate that these are "private" functions, as they're intended
# to be called by our median function but not by other people using our statistics library.
def _median_odd(xs: List[float]) -> float:
  """If len(xs) is odd, the median is the middle element"""
  return sorted(xs)[len(xs) // 2]

def _median_even(xs: List[float]) -> float:
  """If len(xs) is even, it's the average of the middle two elements"""
  sorted_xs = sorted(xs)
  hi_midpoint = len(xs) // 2 # e.g., length 4 => hi_midpoint = 2
  return (sorted_xs[hi_midpoint -1] + sorted_xs[hi_midpoint]) / 2

def median(v: List[float]) -> float:
  """Finds the 'middle-most' value of v"""
  return _median_even(v) if len(v) % 2 == 0 else _median_odd(v)

assert median([1,10,2,9,5]) == 5
assert median([1,9,2,10]) == (2 + 9) / 2

In [0]:
print(median(num_friends))

In [0]:
# Quantiles are a generalization of the median
def quantile(xs: List[float], p: float) -> float:
  """Returns the pth-percentile value in x"""
  p_index = int(p * len(xs))
  return sorted(xs)[p_index]

assert quantile(num_friends, 0.10) == 1
assert quantile(num_friends, 0.25) == 3
assert quantile(num_friends, 0.75) == 9
assert quantile(num_friends, 0.90) == 13

In [0]:
# The mode is the most common value. Sometimes of interest
def mode(x: List[float]) -> List[float]:
  """Returns a list, since there might be more than one mode"""
  counts = Counter(x)
  max_count = max(counts.values())
  return [x_i for x_i, count in counts.items()
            if count == max_count]

assert set(mode(num_friends)) == {1, 6}


In [0]:
# Various ways of looking at dispersion

# Range. In Python, range alredy means something, so we'll name this differently
def data_range(xs: List[float]) -> float:
  return max(xs) - min(xs)

assert data_range(num_friends) == 99

In [0]:
from Chapter_04 import sum_of_squares

# First, a function that centers our mean at 0
def de_mean(xs: List[float]) -> List[float]:
  """Translate xs by subtracting its mean (so the result has mean 0)"""
  x_bar = mean(xs)
  return [x - x_bar for x in xs]

# Variance
def variance(xs: List[float]) -> float:
  """Almost the average squared deviation from the mean"""
  n = len(xs)
  assert n >= 2, "variance requires at least two elements"
  deviations = de_mean(xs)
  return sum_of_squares(deviations) / (n-1)
  
assert 81.54 < variance(num_friends) < 81.55


In [0]:
# Variance has units of the square of the original units, so we usually look at std. dev.
import math

def standard_deviation(xs: List[float]) -> float:
  """The standard deviation is the square root of the variance"""
  return math.sqrt(variance(xs))

assert 9.02 < standard_deviation(num_friends) < 9.04

In [0]:
# Both range and std. dev. have the outlier problem. So a more robust metric 
# computes the difference between the 75th and 25th percentile value:
def interquartile_range(xs: List[float]) -> float:
  """Returns the difference between the 75%-ile and 25%-ile"""
  return quantile(xs, 0.75) - quantile(xs, 0.25)

In [0]:
# We want to see if number of friends is correlated to how long you are on the site
# We'll look at covariance, which measures how two variables vary in tandem from their means
# Copying this data from Grus
daily_minutes = [1,68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]
daily_hours = [dm / 60 for dm in daily_minutes]

def covariance(xs: List[float], ys: List[float]) -> float:
  assert len(xs) == len(ys), "xs and ys must have the same number of elements"
  return Chapter_04.dot(de_mean(xs), de_mean(ys)) / (len(xs) - 1)

assert 22.42 < covariance(num_friends, daily_minutes) < 22.43
assert 22.42 / 60 < covariance(num_friends, daily_hours) < 22.43 / 60

In [0]:
# It's hard to tell what "large" covariance is, so it is more common to look at correlation
# that divides out the sandard deviation of both variables. Always between -1 (perfect anticorrelation)
# and 1 (perfect correlation). 0.25 is a relatively weak positive correlation. 
def correlation(xs: List[float], ys: List[float]) -> float:
  """Measures how much xs and ys vary in tandem about their means"""
  stddev_x = standard_deviation(xs)
  stddev_y = standard_deviation(ys)
  if stddev_x > 0 and stddev_y > 0:
    return covariance(xs, ys) / stddev_x / stddev_y
  else:
    return 0 # if no variation, correlation is zero

  assert 0.24 < correlation(num_friends, daily_minutes) < 0.25
  assert 0.24 < correlation(num_friends, daily_hours) < 0.25

In [0]:
# What happens if we drop outliers in our data, since correlation is sensitive to that
outlier = num_friends.index(100)  # index of outlier
num_friends_good = [x
                    for i, x in enumerate(num_friends)
                    if i != outlier]
daily_minutes_good = [x
                      for i, x in enumerate(daily_minutes)
                      if i != outlier]
daily_hours_good = [dm / 60 for dm in daily_minutes_good]

assert 0.57 < correlation(num_friends_good, daily_minutes_good) < 0.58
assert 0.57 < correlation(num_friends_good, daily_hours_good) < 0.58

In [0]:
# Look out for confounding factors like the example given in Chapter 5. You better
# know your data. There are other relationships that correlation can't find. Read
# this section again, as well. And finally, remember that correlation is not
# causation.

